In [82]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [83]:
data = pd.read_csv('../loadDataBot/po_data.csv')

In [84]:
data

,time,symbol_id,open,close,high,low,asset
0,1538464560,1,1.155696,1.155789,1.155789,1.155685,EURUSD
1,1538464620,1,1.155772,1.155779,1.155779,1.155612,EURUSD
2,1538464680,1,1.155793,1.155770,1.155805,1.155653,EURUSD
3,1538464740,1,1.155733,1.155686,1.155776,1.155636,EURUSD
4,1538464800,1,1.155685,1.155620,1.155685,1.155527,EURUSD
...,...,...,...,...,...,...,...
2982132,1716195420,1,1.087950,1.087800,1.087960,1.087780,EURUSD
2982133,1716195480,1,1.087790,1.087840,1.087940,1.087790,EURUSD
2982134,1716195540,1,1.087840,1.087810,1.087900,1.087750,EURUSD
2982135,1716195600,1,1.087810,1.087870,1.087870,1.087680,EURUSD


In [85]:
data.drop(['asset', 'symbol_id'], axis=1, inplace=True)

In [86]:
data['3min_later'] = data.shift(-3)['close']
data['3min_later_time'] = data.shift(-3)['time']
data['time_diff'] = data['3min_later_time'] - data['time']
data = data[data['time_diff'] == 180]
data['3min_later_result'] = 'equal'
data.loc[data['3min_later'] > data['close'], '3min_later_result'] = 'buy'
data.loc[data['3min_later'] < data['close'],'3min_later_result'] = 'sell'
data.drop(['time_diff', '3min_later_time', '3min_later'], axis=1, inplace=True)
data

/var/folders/gx/rx5rzv1541l34p4fsxdg7cb00000gn/T/ipykernel_750/964220802.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['3min_later_result'] = 'equal'
/var/folders/gx/rx5rzv1541l34p4fsxdg7cb00000gn/T/ipykernel_750/964220802.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(['time_diff', '3min_later_time', '3min_later'], axis=1, inplace=True)


,time,open,close,high,low,3min_later_result
0,1538464560,1.155696,1.155789,1.155789,1.155685,sell
1,1538464620,1.155772,1.155779,1.155779,1.155612,sell
2,1538464680,1.155793,1.155770,1.155805,1.155653,sell
3,1538464740,1.155733,1.155686,1.155776,1.155636,sell
4,1538464800,1.155685,1.155620,1.155685,1.155527,sell
...,...,...,...,...,...,...
2982129,1716195240,1.088090,1.088050,1.088090,1.087940,sell
2982130,1716195300,1.088050,1.087940,1.088050,1.087930,sell
2982131,1716195360,1.087940,1.087940,1.087960,1.087920,sell
2982132,1716195420,1.087950,1.087800,1.087960,1.087780,buy


In [87]:
def rma(x, window):
    """
    Calculate an Exponential Moving Average over a series `x` with a window `window`.
    """
    return x.ewm(span=window, adjust=False).mean()

def rsi(series, period=14):
    """
    Calculate Relative Strength Index (RSI) of a given Pandas series and period length.
    """
    delta = series.diff()  # Change between prices
    up = delta.clip(lower=0)  # Only keep the positive differences (price increases)
    down = -delta.clip(upper=0)  # Only keep the negative differences, make them positive
    
    # Calculate the Exponential Moving Average (RMA) for upward and downward changes
    gain = rma(up, period)
    loss = rma(down, period)
    
    rs = gain / loss  # Relative Strength
    rsi = 100 - (100 / (1 + rs))  # Relative Strength Index
    
    return rsi

def ema(x, window):
    """
    Calculate Exponential Moving Average (EMA) similar to the TradingView Pine Script version.
    """
    return x.ewm(alpha=(2/(window+1)), adjust=False).mean()

def sma(x, window):
    """
    Calculate Simple Moving Average (SMA) using pandas.
    """
    return x.rolling(window).mean()

def macd(x, fast_length, slow_length, is_sma = False):
    fast_ma = sma(x, fast_length) if is_sma else ema(x, fast_length)
    slow_ma = sma(x, slow_length) if is_sma else ema(x, slow_length)
        
    return fast_ma - slow_ma

def macd_signal(macd, window, is_sma=False):
    return sma(macd, window) if is_sma else ema(macd, window)

def bollinger_bands(x, window, mult=2):
    basis = sma(x, window)
    
    dev = mult * x.rolling(window).std()
    
    upper = basis + dev
    lower = basis - dev
    
    return basis, upper, lower

def stochastique(x, window):
    return 100 * (x - x.rolling(window).min()) / (x.rolling(window).max() - x.rolling(window).min())
    
def stochastique_signal(x, window):
    return x.rolling(window).mean()

def sen(x, window):
    return (x.rolling(window).max() + x.rolling(window).min()) / 2

def senko_span_A(tekan_sen, kujin_sen, shift):
    return ((tekan_sen + kujin_sen) / 2).shift(shift)

def senko_span_B(x, window, shift):
    return ((x.rolling(window).min() + x.rolling(window).max()) / 2).shift(shift)

In [88]:
# bollinger bands
data['bb_basis'], data['bb_upper'], data['bb_lower'] = bollinger_bands(data[['close']], 20)

# macd
data['macd_ema'] = macd(data[['close']], 12, 26)
data['macd_sma'] = macd(data[['close']], 12, 26, True)
data['macd_signal_ema'] = macd_signal(data[['macd_ema']], 9)
data['macd_signal_sma'] = macd_signal(data[['macd_sma']], 9, True)

# rsi
data['rsi_9'] = rsi(data[['close']], 9)
data['rsi_14'] = rsi(data[['close']], 14)
data['rsi_28'] = rsi(data[['close']], 28)

# stochastique
data['stochastique'] = stochastique(data[['close']], 14)
data['stochastique_signal'] = stochastique_signal(data[['stochastique']], 14)

# ichimoku
data['tekan_sen'] = sen(data[['close']] ,9)
data['kujin_sen'] = sen(data[['close']] ,26)
data['senko_A'] = senko_span_A(data['tekan_sen'], data['kujin_sen'], 26)
data['senko_B'] = senko_span_B(data[['close']], 52, 26)

/var/folders/gx/rx5rzv1541l34p4fsxdg7cb00000gn/T/ipykernel_750/2099527612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bb_basis'], data['bb_upper'], data['bb_lower'] = bollinger_bands(data[['close']], 20)
/var/folders/gx/rx5rzv1541l34p4fsxdg7cb00000gn/T/ipykernel_750/2099527612.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bb_basis'], data['bb_upper'], data['bb_lower'] = bollinger_bands(data[['close']], 20)
/var/folders/gx/rx5rzv1541l34p4fsxdg7cb00000gn/T/ipykernel_750/2099527612.py

In [89]:
data['bb_basis_hour'], data['bb_upper_hour'], data['bb_lower_hour'] = bollinger_bands(data[['close']], 1200)

# macd
data['macd_ema_hour'] = macd(data[['close']], 720, 1560)
data['macd_sma_hour'] = macd(data[['close']], 720, 1560, True)
data['macd_signal_ema_hour'] = macd_signal(data[['macd_ema_hour']], 540)
data['macd_signal_sma_hour'] = macd_signal(data[['macd_sma_hour']], 540, True)

# rsi
data['rsi_9_hour'] = rsi(data[['close']], 540)
data['rsi_14_hour'] = rsi(data[['close']], 840)
data['rsi_28_hour'] = rsi(data[['close']], 1680)

# stochastique
data['stochastique_hour'] = stochastique(data[['close']], 840)
data['stochastique_signal_hour'] = stochastique_signal(data[['stochastique_hour']], 840)

# ichimoku
data['tekan_sen_hour'] = sen(data[['close']] ,540)
data['kujin_sen_hour'] = sen(data[['close']] ,1560)
data['senko_A_hour'] = senko_span_A(data['tekan_sen_hour'], data['kujin_sen_hour'], 1560)
data['senko_B_hour'] = senko_span_B(data[['close']], 3120, 1560)

/var/folders/gx/rx5rzv1541l34p4fsxdg7cb00000gn/T/ipykernel_750/3610249001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bb_basis_hour'], data['bb_upper_hour'], data['bb_lower_hour'] = bollinger_bands(data[['close']], 1200)
/var/folders/gx/rx5rzv1541l34p4fsxdg7cb00000gn/T/ipykernel_750/3610249001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['bb_basis_hour'], data['bb_upper_hour'], data['bb_lower_hour'] = bollinger_bands(data[['close']], 1200)
/var/folders/gx/rx5rzv1541l34p4fsxdg7cb000

In [90]:
data.dropna(axis=0, inplace=True)
data

/var/folders/gx/rx5rzv1541l34p4fsxdg7cb00000gn/T/ipykernel_750/1877284671.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(axis=0, inplace=True)


,time,open,close,high,low,3min_later_result,bb_basis,bb_upper,bb_lower,macd_ema,...,macd_signal_sma_hour,rsi_9_hour,rsi_14_hour,rsi_28_hour,stochastique_hour,stochastique_signal_hour,tekan_sen_hour,kujin_sen_hour,senko_A_hour,senko_B_hour
4783,1538753520,1.153448,1.153134,1.153652,1.153104,sell,1.152818,1.153460,1.152177,0.000211,...,0.000407,53.341894,52.506340,51.372666,90.048876,41.115899,1.151085,1.151133,1.150196,1.152696
4784,1538753580,1.153088,1.153295,1.153301,1.152897,sell,1.152849,1.153521,1.152177,0.000218,...,0.000405,53.541434,52.655935,51.459164,93.196481,41.154097,1.151085,1.151133,1.150196,1.152696
4785,1538753640,1.153340,1.153131,1.153340,1.153077,sell,1.152869,1.153550,1.152188,0.000208,...,0.000402,53.308343,52.487129,51.365981,89.990225,41.189467,1.151085,1.151133,1.150196,1.152696
4786,1538753700,1.153139,1.153104,1.153354,1.153030,sell,1.152902,1.153560,1.152244,0.000196,...,0.000400,53.270021,52.459376,51.350654,89.462366,41.224629,1.151085,1.151133,1.150196,1.152696
4787,1538753760,1.153115,1.153017,1.153281,1.152999,sell,1.152918,1.153570,1.152266,0.000178,...,0.000397,53.146457,52.369935,51.301271,87.761486,41.256968,1.151085,1.151133,1.150196,1.152696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978639,1715987220,1.086890,1.086890,1.086890,1.086890,equal,1.086933,1.087062,1.086805,-0.000031,...,-0.000556,50.839675,51.116500,50.744054,78.042959,53.170518,1.085860,1.085715,1.087015,1.085295
2978640,1715987280,1.086890,1.086890,1.086890,1.086890,equal,1.086925,1.087043,1.086808,-0.000030,...,-0.000555,50.839675,51.116500,50.744054,78.042959,53.226440,1.085860,1.085715,1.087015,1.085295
2978641,1715987340,1.086890,1.086890,1.086890,1.086890,equal,1.086918,1.087023,1.086813,-0.000029,...,-0.000554,50.839675,51.116500,50.744054,78.042959,53.277740,1.085860,1.085715,1.087015,1.085295
2978642,1715987400,1.086890,1.086890,1.086890,1.086890,equal,1.086913,1.087013,1.086813,-0.000027,...,-0.000553,50.839675,51.116500,50.744054,78.042959,53.305923,1.085860,1.085715,1.087015,1.085295


In [91]:
X = data.drop(['3min_later_result'], axis=1)
y = data['3min_later_result']

In [92]:
X

,time,open,close,high,low,bb_basis,bb_upper,bb_lower,macd_ema,macd_sma,...,macd_signal_sma_hour,rsi_9_hour,rsi_14_hour,rsi_28_hour,stochastique_hour,stochastique_signal_hour,tekan_sen_hour,kujin_sen_hour,senko_A_hour,senko_B_hour
4783,1538753520,1.153448,1.153134,1.153652,1.153104,1.152818,1.153460,1.152177,0.000211,0.000110,...,0.000407,53.341894,52.506340,51.372666,90.048876,41.115899,1.151085,1.151133,1.150196,1.152696
4784,1538753580,1.153088,1.153295,1.153301,1.152897,1.152849,1.153521,1.152177,0.000218,0.000182,...,0.000405,53.541434,52.655935,51.459164,93.196481,41.154097,1.151085,1.151133,1.150196,1.152696
4785,1538753640,1.153340,1.153131,1.153340,1.153077,1.152869,1.153550,1.152188,0.000208,0.000224,...,0.000402,53.308343,52.487129,51.365981,89.990225,41.189467,1.151085,1.151133,1.150196,1.152696
4786,1538753700,1.153139,1.153104,1.153354,1.153030,1.152902,1.153560,1.152244,0.000196,0.000248,...,0.000400,53.270021,52.459376,51.350654,89.462366,41.224629,1.151085,1.151133,1.150196,1.152696
4787,1538753760,1.153115,1.153017,1.153281,1.152999,1.152918,1.153570,1.152266,0.000178,0.000259,...,0.000397,53.146457,52.369935,51.301271,87.761486,41.256968,1.151085,1.151133,1.150196,1.152696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978639,1715987220,1.086890,1.086890,1.086890,1.086890,1.086933,1.087062,1.086805,-0.000031,-0.000060,...,-0.000556,50.839675,51.116500,50.744054,78.042959,53.170518,1.085860,1.085715,1.087015,1.085295
2978640,1715987280,1.086890,1.086890,1.086890,1.086890,1.086925,1.087043,1.086808,-0.000030,-0.000058,...,-0.000555,50.839675,51.116500,50.744054,78.042959,53.226440,1.085860,1.085715,1.087015,1.085295
2978641,1715987340,1.086890,1.086890,1.086890,1.086890,1.086918,1.087023,1.086813,-0.000029,-0.000051,...,-0.000554,50.839675,51.116500,50.744054,78.042959,53.277740,1.085860,1.085715,1.087015,1.085295
2978642,1715987400,1.086890,1.086890,1.086890,1.086890,1.086913,1.087013,1.086813,-0.000027,-0.000049,...,-0.000553,50.839675,51.116500,50.744054,78.042959,53.305923,1.085860,1.085715,1.087015,1.085295


In [93]:
y

4783        sell
4784        sell
4785        sell
4786        sell
4787        sell
           ...  
2978639    equal
2978640    equal
2978641    equal
2978642    equal
2978643    equal
Name: 3min_later_result, Length: 1821598, dtype: object

In [94]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import time
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [96]:
#model = RandomForestClassifier(n_estimators=750, max_depth=70, verbose=4, n_jobs=-1)
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import LabelEncoder

numerical_features = make_column_selector(dtype_include=np.number)

numerical_pipeline = make_pipeline(StandardScaler())

preprocessor = make_column_transformer((numerical_pipeline, numerical_features))

y_tranformer = LabelEncoder()

model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, max_depth=70, verbose=4, n_jobs=-1))

model.fit(X_train, y_tranformer.fit_transform(y_train))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 100building tree 2 of 100
building tree 3 of 100

building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   45.4s


building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60

[Parallel(n_jobs=-1)]: Done  82 tasks      | elapsed:  4.1min


building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  4.9min finished


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x294deb990>)])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=70, n_jobs=-1, verbose=4))])

In [97]:
model.score(X_test, y_tranformer.transform(y_test))

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    5.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    5.9s finished


0.6958519982433026

In [98]:
probas = model.predict_proba(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    5.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    5.9s finished


In [103]:
lose = 0
win = 0
threshold = 0.75
capital = 100
per_trade = 0.01
cote = 0.7
for [long, even, short, result] in np.hstack((probas, y_test.values.reshape(y_test.shape[0], 1))):
    if short > threshold:
        if result == 'buy':
            lose = lose + 1
            capital = capital - capital * per_trade
        elif result == 'sell': 
            win = win + 1
            capital = capital + capital * per_trade * cote
    if long > threshold:
        if result == 'sell':
            lose = lose + 1
            capital = capital - capital * per_trade
        elif result == 'buy': 
            win = win + 1
            capital = capital + capital * per_trade * cote
print(f'lose: {lose}; win: {win}; ratio: {win/(lose+win)}; capital: {capital}')

lose: 4345; win: 29888; ratio: 0.873075687202407; capital: 3.7995873481233626e+73


In [104]:
(win + lose) / len(y_test) * 60

4.510276679841897

In [ ]:
from joblib import dump, load
dump(model, 'model_EURUSD.joblib', compress=9) 

['model_eurusd.joblib']

## Mise en situation côté serveur python

In [ ]:
model2 = load('model_eurusd.joblib') 

In [ ]:
import json

jsonData = None

with open('data.json') as file: 
    jsonData = json.load(file)

In [ ]:
jsonData

In [ ]:
df = pd.DataFrame(jsonData)

df.set_index('date', inplace=True)

df.drop(['session_date', 'start_of_day'], axis=1, inplace=True)

# bollinger bands
df['bb_basis'], df['bb_upper'], df['bb_lower'] = bollinger_bands(df[['close']], 20)

# macd
df['macd_ema'] = macd(df[['close']], 12, 26)
df['macd_sma'] = macd(df[['close']], 12, 26, True)
df['macd_signal_ema'] = macd_signal(df[['macd_ema']], 9)
df['macd_signal_sma'] = macd_signal(df[['macd_ema']], 9, True)

# rsi
df['rsi_9'] = rsi(df[['close']], 9)
df['rsi_14'] = rsi(df[['close']], 14)
df['rsi_28'] = rsi(df[['close']], 28)

df.dropna(axis=0, inplace=True)

In [ ]:
df

,open,high,low,close,volume,bb_basis,bb_upper,bb_lower,macd_ema,macd_sma,macd_signal_ema,macd_signal_sma,rsi_9,rsi_14,rsi_28
date,,,,,,,,,,,,,,,
2024-02-11T22:32:00Z,1.07844,1.07846,1.07844,1.07846,2,1.078421,1.078509,1.078333,0.000014,0.000036,0.000011,0.000012,64.741109,61.761420,58.289268
2024-02-11T22:35:00Z,1.07846,1.07848,1.07844,1.07848,5,1.078430,1.078501,1.078359,0.000016,0.000035,0.000012,0.000013,75.546617,68.712645,62.019256
2024-02-11T22:36:00Z,1.07848,1.07854,1.07848,1.07854,11,1.078439,1.078518,1.078360,0.000023,0.000037,0.000014,0.000014,88.622271,80.796563,70.515225
2024-02-11T22:37:00Z,1.07853,1.07858,1.07850,1.07858,13,1.078448,1.078547,1.078349,0.000030,0.000044,0.000017,0.000017,92.129396,85.195044,74.585906
2024-02-11T22:38:00Z,1.07862,1.07866,1.07862,1.07864,8,1.078460,1.078588,1.078332,0.000041,0.000051,0.000022,0.000020,95.012162,89.397971,79.210197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-01T11:21:00Z,1.07834,1.07836,1.07833,1.07833,21,1.078285,1.078601,1.077968,0.000065,0.000166,0.000067,0.000079,42.624041,49.207106,53.913394
2024-04-01T11:22:00Z,1.07833,1.07836,1.07833,1.07835,14,1.078293,1.078607,1.077978,0.000061,0.000156,0.000066,0.000077,47.839473,51.970526,55.197161
2024-04-01T11:23:00Z,1.07834,1.07838,1.07831,1.07834,30,1.078304,1.078606,1.078003,0.000056,0.000137,0.000064,0.000074,45.267722,50.388925,54.383612


In [ ]:
model2.predict_proba(df)

[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done  58 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 181 tasks      | elapsed:    0.6s
[Parallel(n_jobs=20)]: Done 352 tasks      | elapsed:    1.6s
[Parallel(n_jobs=20)]: Done 573 tasks      | elapsed:    2.9s
[Parallel(n_jobs=20)]: Done 750 out of 750 | elapsed:    3.9s finished


array([[0.472     , 0.02933333, 0.49866667],
       [0.432     , 0.03466667, 0.53333333],
       [0.424     , 0.10666667, 0.46933333],
       ...,
       [0.36      , 0.06      , 0.58      ],
       [0.44      , 0.06533333, 0.49466667],
       [0.46266667, 0.028     , 0.50933333]])